# Combinando Datasets: Concat y Append

Cuando realizamos un análisis de datos, es normal que queramos analizar más información de la que tenemos en una sola fuente, por lo que nos interesará añadir más datos provenientes de otras distintas, como hemos visto en el temario de SQL.

Estas operaciones pueden implicar cualquier cosa, desde la concatenación sencilla de dos conjuntos de datos diferentes, hasta combinaciones y combinaciones más complicadas propias de bases de datos que controlan correctamente las superposiciones entre los conjuntos de datos.

Los objetos ``Series`` y ``DataFrame`` que implementa Pandas se han diseñado con este tipo de operación en mente, así como diversas funciones y métodos que hacen que este tipo de lucha de datos sea rápido y sencillo.

A continuación, veremos la concatenación simple de ``Series`` y ``DataFrame`` con la función ``pd.concat``; y tras ello, estudiaremos ``merges`` y ``joins``, que serán operaciones para combinar fuentes de datos de un carácter algo más complejo.

Comenzamos importando las librerías básicas:

In [2]:
import pandas as pd
import numpy as np

Para ahorrarnos código en el futuro, nos crearemos una función que simplemente cree un ``DataFrame`` con la forma que veremos a continuación y que nos servirá para entender los diferentes ejemplos:

In [3]:
def make_df(cols, ind):
    """Función para crear de forma rápida un DataFrame para usar en los ejemplos"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
df = make_df('ABC', range(3))
df

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


Además, reutilizaremos la función del otro día que nos permitía imprimir bonito por pantalla más de un ``DataFrame`` a la vez, y a la que llamaremos pasándole como argumento diferentes cadenas de texto que dieran como salida un DataFrame:

In [4]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

In [5]:
display("df", "df.iloc[:2]", "df[['A']]")

df
    A   B   C
0  A0  B0  C0
1  A1  B1  C1
2  A2  B2  C2

df.iloc[:2]
    A   B   C
0  A0  B0  C0
1  A1  B1  C1

df[['A']]
    A
0  A0
1  A1
2  A2

## Recordatorio: Concatenación de Arrays de NumPy

La concatenación de ``Series`` y ``DataFrames`` es muy similar a la concatenación de arrays de NumPy, para lo que utilizábamos la función ``np.concatenate``.

Son esto en mente, podemos entender la concatenación de ``DataFrames``, pues hemos visto que, en el fondo, sus valores son arrays de NumPy. Veamos un par de ejemplos de la concatenación de NumPy:

In [6]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

El primer argumento es una lista (o tupla) con los arrays que queremos concatenar.

Adicionalmente, habíamos visto que esta función tiene un parámetro ``axis`` que nos permitirá especificar en qué eje realizar la concatenación, es decir, de forma horizontal (``axis=1``) o vertical (``axis=0``), niveles que se verán con mayor claridad cuando lo usemos con pandas:

In [7]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x])

array([[1, 2],
       [3, 4],
       [1, 2],
       [3, 4]])

## Concatenación simple con pandas: ``pd.concat``

Para concatenar objetos con Pandas, utilizaremos la función ``pd.concat()``, cuya sintaxis es similar a la vista para arrays de NumPy, pero con una mayor variedad de opciones:

```python
# Signature in Pandas v0.18
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
```

Además, pese a que tenga muchos parámetros, por defecto actuará como la función de NumPy, pero aceptando objetos ``Series`` y ``DataFrame``:

In [8]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

Pero no sólo sirve para concatenar, también lo podemos utilizar con ``DataFrame``:

In [9]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
display('df1', 'df2', 'pd.concat([df1, df2])')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


Como hemos comentado anteriormente, al igual que lo que hemos visto con NumPy, podemos utilizar más de un eje para hacer la concatenación: vertical (``axis=0`` o ``axis='rows'``, eje por defecto) u horizontal (``axis=1`` o ``axis='columns'``).

Veámoslo con un ejemplo:

In [10]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4])", "pd.concat([df3, df4], axis='rows')", "pd.concat([df3, df4], axis='columns')")

df3
    A   B
0  A0  B0
1  A1  B1

df4
    C   D
0  C0  D0
1  C1  D1

pd.concat([df3, df4])
     A    B    C    D
0   A0   B0  NaN  NaN
1   A1   B1  NaN  NaN
0  NaN  NaN   C0   D0
1  NaN  NaN   C1   D1

pd.concat([df3, df4], axis='rows')
     A    B    C    D
0   A0   B0  NaN  NaN
1   A1   B1  NaN  NaN
0  NaN  NaN   C0   D0
1  NaN  NaN   C1   D1

pd.concat([df3, df4], axis='columns')
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1

### Duplicidad de índices

Si bien la forma de operar de ``pd.concat()`` es similar a la vista para ``np.concat()``, la funcionalidad que aporta gracias a sus parámetros es mucho mayor. Una de estas diferencias reside en que la versión de Pandas mantiene los índices incluso si el resultado final da lugar a un elemento con índices repetidos, como es posible que te hayas fijado en el ejemplo anterior.

Veamos otro donde lo pongamos de manifiesto, donde creamos dos ``DataFrame`` con el mismo índice y concatenamos de forma vertical

In [11]:
x = make_df('AB', [0, 1])
y = make_df('AB', [0, 1])
display('x', 'y', 'pd.concat([x, y])')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A0,B0
1,A1,B1
,A,B
0,A0,B0
1,A1,B1
0,A0,B0
1,A1,B1


Fíjate que, pese a que tengamos índices repetidos, el resultado es totalmente válido y no da lugar a errores. Sin embargo, puede que no sea lo que queremos y que termine desembocando a un comportamiento inesperado de nuestro script.

Por eso, veremos ciertas formas de gestionarlo:

#### Capturando duplicidad de índices como errores

Una opción podría ser detectar la duplicidad de índices como un error, útil en el caso de que queramos detener el programa (o no, si capturamos el error), ya que generará una excepción al detectar este fenómeno. Para ello, podemos utilizar el parámetro ``verify_integrity`` de la función ``pd.concat()``, asignándole el valor ``True``.

Ejemplo donde se lanza una excepción al detectar esa duplicidad de índices:

In [12]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


#### Ignorando el índice

A veces, los índices originales de los objetos que queremos combinar no nos importa demasiado, lo que nos importa son los datos en sí. En este caso, podemos ignorar el índice en la concatenación, lo que hará que el resultado de salida sea uno nuevo basado en las posiciones que ocupan en este objeto resultante.

Para hacerlo, activaremos el parámetro ``ignore_index``, vomo en el siguiente ejemplo:

In [13]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A0,B0
1,A1,B1
,A,B
0,A0,B0
1,A1,B1
2,A0,B0
3,A1,B1


#### Añadiendo índices de más de una dimensión (multi-index)

Otra opción sería mantener los índices de origen y la funete de la que proviene, creando un multi-index de 2 dimensiones, donde la primera se corresponderá con el nombre que asignaremos al objeto en el cruce, y la segunda lo hará con el proipio índice, es decir, el índice del resultado será jerárquico.

Para ello, deberemos especificar, mediante el parámetro ``keys``, el nombre que queremos asignar a ese primer nivel de índices (que hará referencia a cada uno de los objetos que utolicemos para la concatenación). Este parámetro recibirá generalmente una lista de strings, aunque podría ser otro tipo de iteradores y valores, como un ``range``, siempre y cuando se asegure la longitud igual a los elementos utilizados en el cruce:

In [14]:
display('x', 'y', "pd.concat([x, y], keys=range(2))")

x
    A   B
0  A0  B0
1  A1  B1

y
    A   B
0  A0  B0
1  A1  B1

pd.concat([x, y], keys=range(2))
      A   B
0 0  A0  B0
  1  A1  B1
1 0  A0  B0
  1  A1  B1

In [15]:
df = pd.concat([x, y], keys=['x', 'y'])

In [16]:
# Aunque lo veremos en el futuro, voy a resolver vuestra curiosidad, podremos acceder por nombre a una fila en concreto de un DataFrame con índice jerárquico 
#del siguiente modo:
df.loc[('y', 0)] #

A    A0
B    B0
Name: (y, 0), dtype: object

### concatenación con ``join``

Hasta ahora, lo que hemos hecho es combinar ``DataFrames`` con las mismas columnas. Sin embargo, en la práctica veremos que esto cambia, pues lo más normal es que fuentes diferentes no tengan las mismas columnas. Para ello, ``pd.concat()`` nos ofrece varias opciones.

Considerando una combinación en la que los ``DataFrames`` tienen alguna columna en común y otras no:

In [17]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6])')

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6])
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4

Por defecto, los registros sin datos se rellenan con NaN, valor que ya hemos discutido con anterioridad.

Para cambiar esto, podemos utilizar el parámetro ``join``, donde especificaremos qué tipo de union se va a realizar. Por defecto, el parámetro ``join='outer'``, que hará la unión de las columnas de ambos ``DataFrames``, lo que da lugar a rellenar con NaN. Sin embargo, este parámetro podemos cambiarlo por otras opciones, como ``join='inner'``, que nos devolverá como resultado la intersección de las columnas:

In [18]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner')")

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], join='inner')
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4

Esta solo es una de las configuraciones que nos permite ``pd.concat()``, pero tiene muchas más posibilidades, asi que si tienes alguna idea a la hora de realizar un cruce, consulta la librería. Puede que esa funcinoalidad ya esté implmentada por defecto y te ahorre algún que otro quebradero de cabeza.

### Ejercicio

Supongamos que tenemos los DataFrames ``dfe1, dfe2, dfe3`` y ``dfe4``:
1. Realiza la concatenación de ``dfe1`` y ``dfe2``, con y sin obtener duplicidad de índice. En el caso que no tengamos duplicidad de índices, queremos que solo sea de una dimensión
2. Añade las 10 primeras filas de ``dfe3`` a la concatenación anterior sin duplicados. En este caso, realiza el cruce de tal forma que si encuentras duplicados en la salida salte un error. Si salta, cambia el índice (a mano) de lo que creas necesario para que no salte.
3. Ahora añade 15 registros aleatorios de ``dfe4`` al resultado anterior, pero quédate con las columnas comunes
4. Ahora, crea un ``Series`` combinando los 10 primeros registros de las columnas 'C' de ``dfe3`` y 'D' de ``dfe4``, haciendo un join de ``Series``

In [19]:
dfe1 = make_df('ABC', range(20))
dfe2 = make_df('ABC', np.random.randint(0, 30, size=20))
dfe3 = make_df('CQWX', range(100)).iloc[40:]
dfe4 = make_df('ADQM', range(30)).iloc[10:]

# 1. 
print("Ej. 1:")
dfc1 = pd.concat([dfe1, dfe2])
dfc12 = pd.concat([dfe1, dfe2], ignore_index=True)
print(dfc12)

# 2.
print("\n\nEj. 2:")
dfc2 = pd.concat([dfc12, dfe3.iloc[:10]], ignore_index=True)
print(dfc2)

# 3.
print("\n\nEj. 3:")
dfe4_aux = dfe4.iloc[np.random.choice(np.arange(len(dfe4)), size=15, replace=False)].sort_index()
print(pd.concat([dfc2, dfe4_aux], join='inner'))

# 4.
print("\n\nEj. 4:")
ser31 = dfe3['C'].iloc[:10]
ser41 = dfe4['D'].iloc[:10]
ser34 = pd.concat([ser31, ser41])
print(ser34)



Ej. 1:
      A    B    C
0    A0   B0   C0
1    A1   B1   C1
2    A2   B2   C2
3    A3   B3   C3
4    A4   B4   C4
5    A5   B5   C5
6    A6   B6   C6
7    A7   B7   C7
8    A8   B8   C8
9    A9   B9   C9
10  A10  B10  C10
11  A11  B11  C11
12  A12  B12  C12
13  A13  B13  C13
14  A14  B14  C14
15  A15  B15  C15
16  A16  B16  C16
17  A17  B17  C17
18  A18  B18  C18
19  A19  B19  C19
20  A21  B21  C21
21  A27  B27  C27
22  A24  B24  C24
23  A21  B21  C21
24   A9   B9   C9
25   A3   B3   C3
26  A28  B28  C28
27  A29  B29  C29
28  A10  B10  C10
29  A19  B19  C19
30  A22  B22  C22
31   A4   B4   C4
32   A4   B4   C4
33  A15  B15  C15
34  A22  B22  C22
35  A10  B10  C10
36   A0   B0   C0
37  A20  B20  C20
38   A3   B3   C3
39   A0   B0   C0


Ej. 2:
      A    B    C    Q    W    X
0    A0   B0   C0  NaN  NaN  NaN
1    A1   B1   C1  NaN  NaN  NaN
2    A2   B2   C2  NaN  NaN  NaN
3    A3   B3   C3  NaN  NaN  NaN
4    A4   B4   C4  NaN  NaN  NaN
5    A5   B5   C5  NaN  NaN  NaN
6    A6   B6   

### El método ``append()``

Así como tenemos la función ``pd.concat()``, también tenemos un método que hace lo mismo pero cambiando un poco la dinámica, lo que a veces nos agiliza la codificación: el método ``append()``.

En este caso, estamos hablando de un método de los ``DataFrames``, por eso se llama como un método de los mismos.

Para que entendamos a qué nos referimos, la sentencia ``pd.concat([df1, df2])`` es equivalente a escribir ``df1.append(df2)``:

In [20]:
display('df1', 'df2', 'pd.concat([df1, df2])', 'df1.append(df2)')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


# Combinando Datasets: ``merge`` y ``join``

Una característica esencial que ofrece Pandas son sus operaciones de comniación ``merge`` y ``join``, las cuales ofrecen un alto rendimiento en memoria.
Al que tenga experiencia con bases de datos, esto le resultará familiar. Sin embargo, si no has tenido contacto con este mundo, tampoco te debes preocupar, pues ahora lo cubriremos.

La interfaz principal para esto es la función ``pd.merge``, de la veremos algunos ejemplos.

## Álgebra relacional

La función de ``pd.merge()`` se basa en un subconjunto de lo que se conoce como álgebra relacional, que es un conjunto formal de reglas para manipular datos relacionales, formando la base conceptual de las operaciones disponibles en la mayoría de bases de datos.

La principal característica del enfoque del álgebra relacional es que propone varias operaciones primitivas, que se pueden combinar para obtener funcionalidades más complicadas en cualquier conjunto de datos.

Con este conjunto de operaciones fundamentales, el cual está implementado de manera eficiente en una base de datos (que en nuestro caso será un ``DataFrame``), se puede realizar una amplia gama de operaciones compuestas para cubrir funcionalidades de elevada complejidad.

Pandas implementa varios de estos bloques de construcción fundamentales mediante la función ``pd.merge()`` y el método ``join()``, disponible tanto para  ``Series`` como para ``Dataframe``.

Como veremos, con estas herramientas podremos combinar datos de diversas funetes para enriquecer nuestros datasets.

## Categorías de ``join``

La función ``pd.merge()`` implementa diferentes tipos de joins: *uno-a-uno*, *varios-a-uno*, y *varios-a-varios*.

Estos 3 tipos se invocarán de la misma forma mediante la función ``pd.merge()``, siendo utilizado uno u otro de forma implícita dependiendo de los datos que estemos combinando, pues será el propio merge el que identifique qué tipo de relación se da entre los datos.

A continuación, explicaremos brevemente cada uno de ellos:

### Join Uno-a-uno

Quizás el más simple de los 3 es el join uno-a-uno, el cual es muy parecido a lo que hemos visto para concatenar DataFrames horizontalmente

A continuación, veremos un ejemplo donde juntamos información sobre empleados que está distribuida en 2 datasets diferentes:

In [21]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


Para combinar la info de los ``DataFrame``, utilizamos la función ``pd.merge()``:

In [22]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


La función ``pd.merge()`` identifica que cada ``DataFrame`` tiene una columna "employee", y automáticamente la utiliza como clave del cruce.
El resultado del cruce es un nuevo ``DataFrame`` que combina la información de ambos datasets.

Fíjate que el orden de las entradas en cada columna no es un factor determinante a la hora de hacer el cruce, pues el orden de los datos en la columna "employee" del ``df1`` es distinto al visto en ``df2``. De hecho, si nos fijamos bien, veremos que se ha mantenido el orden del primer ``DataFrame`` (``df1``).

Además, ten en mente que los cruces generalmente descartarán el índice, pues se basarán en los valores de las columnas para realizar los cruces. Sin embargo, sí que habra algunos casos donde serán importantes, como los joins ``left_index`` o ``right_index``, los cuales se discutirán en el futuro.

### Join Varios-a-uno

Los cruces Varios-a-uno son joins en los que uno de las dos columnas que participan en el cruce no tiene valores únicos, es decir, tiene duplicados (de ahí el varios), mientras que la otra tiene valores únicos.

Para la mayoría de casos, la salida mantendrá estos elementos duplicados, que es lo que buscamos al realizar un cruce de este tipo:

In [23]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

Como puedes observar, el ``DataFrame`` de salida tiene una nueva columna ("supervisor"), que replica los datos para aquellos grupos que se repitan.

### Join Varios-a-varios

Las uniones varios-a-varios son un poco confusas conceptualmente, pero están bien definidas.

Si la columna de cruce en ambos ``DataFrames`` contiene duplicados, el resultado es una combinación de muchos a muchos. En este caso, la salida será mayor que los ``DataFrames`` utilizados para realizar el cruce, por lo que este tipo de operaciones tienen que estar muy bien controladas. Al usar grandes conjuntos de datos, obtendremos una salida como el producto vectorial de los cruces, lo que hará que el resultado crezca de forma incontrolada, llegando a niveles que nos consumirían todos los recursos de la máquina donde lo estemos ejecutando, pudiendo llegar a no completarse nunca y, en caso de conseguirlo, obteniendo algo que no deseamos.

Veamos cómo funcionan mediante un ejemplo:

  - Imaginemos que tenemos un ``DataFrame`` que muestra una o más habilidades asociadas con un grupo en particular.
Al realizar un join varios-a-varios, podemos recuperar las habilidades asociadas con cualquier persona individual:

In [24]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


Estos tres tipos de combinaciones se pueden usar con otras herramientas de Pandas para implementar una amplia gama de funcionalidades.

En la práctica, los conjuntos de datos rara vez son tan limpios como los que estamos utilizando para estos ejemplos. Por ello, en la siguiente sección, consideraremos algunas de las opciones proporcionadas por ``pd.merge()`` que nos permitirán ajustar cómo funcionan las operaciones de unión.

## Especificación de la clave de la unión

Hemos visto el comportamiento por defecto de la función ``pd.merge()``, el cual es capaz de utilizar una o varias columnas para buscar el cruce entre 2 ``DataFrames``.
Sin embargo, algunas veces los nombres de las columnas del cruce no son iguales, para lo que ``pd.merge()`` nos ofrece una serie de herramientas.

### El parámetro ``on``

El cruce más sencillo, en el caso de que la/s columna/s de cruce se llamen igual, para lo cual podemos establecer cuál será la columna de cruce mediante el parámetro ``on``, el cual toma el nombre de la columna de cruce (o una lista de nombres si hay más de una):

In [25]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

Esta opción solo funcionará si ambos  ``DataFrames`` tienen la/s columna/s especificada/s..

### Los parámetros ``left_on`` y ``right_on``

A veces, tendremos que unir 2 ``DataFrames`` en base a columnas que se llamen distinto en cada uno.

Por ejemplo, podemos tener un dataset con una columna de "employee", que tenga los nombres de los empleados, que queramos cruzar con otra donde la columna con los nombres de los empleados se llame "name". En este caso, podemos utilizar ``left_on`` y ``right_on`` para especificar el nombre de la columna en cada ``DataFrame``:

In [26]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

Como podrás observar, este cruce genera una columna redundante, ya que tendremos cada una de las columnas del cruce. Para quitarla, podemos utilizar el método ``drop()``:

In [27]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### Los parámetros ``left_index`` y ``right_index``

A veces, en lugar de hacer un cruce por una columna, lo que querremos será realizar un cruce por los índices, por ejemplo:

In [28]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


En este caso, podemos especificar que queremos utilizar como clave de cruce el índice tanto a la izquierda como a la derecha, para lo que utilizaremos los parámetros ``left_index`` y/o ``right_index`` al llamar a la función ``pd.merge()``:

In [29]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Por conveniencia, al igual que el método ``append()`` respecto a la función ``pd.concat()``, existe un método equivalente a la unión mediante índice de la función ``pd.merge(left_index=True, right_index=True)``: el método ``join()``:

In [30]:
display('df1a', 'df2a', 'df1a.join(df2a)')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


En el caso de querer realizar un cruce combinando un índice de un ``DataFrame`` con una columna del otro, podemos mezclar los parámetros ``left_index`` con ``right_on`` o ``left_on`` con ``right_index``:

In [31]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')")

df1a
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1a, df3, left_index=True, right_on='name')
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000

Lo que hemos visto hasta ahora es para la combinación de 2 ``DataFrames``, sin embargo, la mayoría de lo que hemos visto es extendible a varios ``DataFrames`` de forma bastante intuitiva. No obstante, te recomiendo consultar la documentación:  ["Merge, Join, and Concatenate" section](http://pandas.pydata.org/pandas-docs/stable/merging.html).

## Especificando la aritmética de cruce de la unión

En todos los ejemplos anteriores, hemos pasado por alto una consideración importante al realizar una combinación: la aritmética utilizada en el cruce.
Esto se pone de manifiesto cuando aparece un valor en una columna clave pero no en la otra.

Veámoslo con un ejemplo:

In [32]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

Aquí hemos hecho la unión de dos conjuntos de datos que tienen una sola entrada de "nombre" en común: Mary.

Por defecto, el resultado contiene la intersección de los dos conjuntos de entradas, lo que se conoce como "inner join". Esto lo podemos especificar explícitamente mediante el parámetro ``how``, el cual por defecto toma el valor ``"inner"``:

In [33]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


Otras opciones para el parámetro ``how`` son ``'outer'``, ``'left'``, y ``'right'``:

  - El cruce "outer join" devuelve un ``DataFrame`` con la unión de las columnas de los 2 ``DataFrames`` de entrada, rellenando los valores faltantes con NaN:

In [34]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='outer')
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer

Los cruces "left join" y "right join" devuelve la unión sobre las columnas de la izquierda o de la derecha, respectivamente. Esto significa que, en el caso del cruce "left join", el ``DataFrame`` de salida tendrá únicamente los registros cuyos valores de la columna de cruce estén en el ``DataFrame`` de la izquierda, no incluyendo ningún registro cuyo valor en la columna de cruce esté contenido exclusivamente en el de la derecha.

Veamos un ejemplo:

In [35]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

In [36]:
display('df6', 'df7', "pd.merge(df6, df7, how='right')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='right')
     name   food drink
0    Mary  bread  wine
1  Joseph    NaN  beer

Como podemos comprobar, en el caso de que un valor de cruce no esté incluido en la columna del otro, se rellena con NaN.

Todas estas opciones pueden ser utilizadas con cualquier tipo de unión.

## Superposición de nombres de columnas: el parámetro ``suffixes``

Finalmente, podríamos llegar al caso en que se produzca un conflicto entre las columnas de los ``DataFrames`` de entrada, es decir, que tenga columnas en el ``DataFrame`` de la izquierda con el mismo nombre de alguna de la derecha.

Por ejemplo:

In [124]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on="name")')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name")
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

En este caso, lo que hace el ``pd.merge()``, que lo detecta automáticamente, es renombrar las columnas conflictivas con un sufijo: las que originalmente estaban en el ``DataFrame`` de la izquierda con el sufijo "_x", y el de la derecha con el sufijo "_y". De este modo, evita el conflicto con las columnas con mismo nombre, ya que un ``DataFrame`` no puede tener 2 columnas iguales.

Si queremos modificar estos sufijos por algo que nos interese, podemos hacerlomediante el parámetro ``suffixes``, que recibirá una lista con un par de strings (aunque también aceptaría la mayoría de iteradores con tamaño 2):

In [126]:
display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])', 'pd.merge(df8, df9, on="name", suffixes=range(2))')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

pd.merge(df8, df9, on="name", suffixes=range(2))
   name  rank0  rank1
0   Bob      1      3
1  Jake      2      1
2  Lisa      3      4
3   Sue      4      2

Estos sufijos funcionan en cualquiera de los posibles patrones de combinación, así como en el caso de que haya varias columnas superpuestas.

Podemos profundizar sobre los cruces esto en la documentación ["Merge, Join and Concatenate"](http://pandas.pydata.org/pandas-docs/stable/merging.html).

Y si quieres más información sobre los joins, pero a un nivel más genérico, basándose en explicaciones de SQL, puedes acceder a [este enlace](https://blog.codinghorror.com/a-visual-explanation-of-sql-joins/)

### Ejercicio 2

Supongamos que tenemos los DataFrames ``dfe1, dfe2`` y ``dfe3``, donde las columnas que se deben usar para cruzar los datos son C y A, respectivamente, aunque existan otras comunes:
1. Realiza el cruce entre los 3 mediante ``merge`` para quedarnos con todos los registros
2. Realiza el cruce mediante ``join`` (deberás buscar cómo aplicar los sufijos en el [join](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html))
3. Realiza el cruce para quedarnos únicamente con los registros de ``dfe1``
4. Realiza el cruce para quedarnos únicamente con los registros de ``dfe3``
5. Quédate solo con aquellos registros que crucen perfectamente, es decir, que no nos añadan NaNs

In [85]:
dfe1 = make_df('ABC', range(20))
dfe2 = make_df('CQWX', range(100)).iloc[:40]
dfe3 = make_df('ADQM', range(30)).iloc[10:]

# 1. 
print("Ej. 1:")
cruce1_1 = pd.merge(dfe1, dfe2, how='outer', on='C')
cruce1_2 = pd.merge(cruce1_1, dfe3, how='outer', on='A')
cruce1_2

Ej. 1:


,A,B,C,Q_x,W,X,D,Q_y,M
0,A0,B0,C0,Q0,W0,X0,NaN,NaN,NaN
1,A1,B1,C1,Q1,W1,X1,NaN,NaN,NaN
2,A2,B2,C2,Q2,W2,X2,NaN,NaN,NaN
3,A3,B3,C3,Q3,W3,X3,NaN,NaN,NaN
4,A4,B4,C4,Q4,W4,X4,NaN,NaN,NaN
5,A5,B5,C5,Q5,W5,X5,NaN,NaN,NaN
6,A6,B6,C6,Q6,W6,X6,NaN,NaN,NaN
7,A7,B7,C7,Q7,W7,X7,NaN,NaN,NaN
8,A8,B8,C8,Q8,W8,X8,NaN,NaN,NaN
9,A9,B9,C9,Q9,W9,X9,NaN,NaN,NaN


In [86]:
# 2.
print("Ej. 2:")
dfe1_2 = dfe1.copy().set_index("C")
dfe2_2 = dfe2.copy().set_index("C")
cruce2_1 = dfe1_2.join(dfe2_2, how='outer')
cruce2_1 = cruce2_1.set_index("A")
dfe2_3 = dfe3.copy().set_index("A")
cruce2_2 = cruce2_1.join(dfe2_3, how='outer', lsuffix='_x', rsuffix='_y')
cruce2_2['A'] = cruce2_2.index
cruce2_2 = cruce2_2.reset_index(drop=True)
cruce2_2



Ej. 2:


,B,Q_x,W,X,D,Q_y,M,A
0,B0,Q0,W0,X0,NaN,NaN,NaN,A0
1,B1,Q1,W1,X1,NaN,NaN,NaN,A1
2,B10,Q10,W10,X10,D10,Q10,M10,A10
3,B11,Q11,W11,X11,D11,Q11,M11,A11
4,B12,Q12,W12,X12,D12,Q12,M12,A12
5,B13,Q13,W13,X13,D13,Q13,M13,A13
6,B14,Q14,W14,X14,D14,Q14,M14,A14
7,B15,Q15,W15,X15,D15,Q15,M15,A15
8,B16,Q16,W16,X16,D16,Q16,M16,A16
9,B17,Q17,W17,X17,D17,Q17,M17,A17


In [87]:
# 3. 
print("Ej. 3:")
cruce3_1 = pd.merge(dfe1, dfe2, how='left', on='C')
cruce3_2 = pd.merge(cruce3_1, dfe3, how='left', on='A')
cruce3_2

Ej. 3:


,A,B,C,Q_x,W,X,D,Q_y,M
0,A0,B0,C0,Q0,W0,X0,NaN,NaN,NaN
1,A1,B1,C1,Q1,W1,X1,NaN,NaN,NaN
2,A2,B2,C2,Q2,W2,X2,NaN,NaN,NaN
3,A3,B3,C3,Q3,W3,X3,NaN,NaN,NaN
4,A4,B4,C4,Q4,W4,X4,NaN,NaN,NaN
5,A5,B5,C5,Q5,W5,X5,NaN,NaN,NaN
6,A6,B6,C6,Q6,W6,X6,NaN,NaN,NaN
7,A7,B7,C7,Q7,W7,X7,NaN,NaN,NaN
8,A8,B8,C8,Q8,W8,X8,NaN,NaN,NaN
9,A9,B9,C9,Q9,W9,X9,NaN,NaN,NaN


In [88]:
# 4. 
print("Ej. 4:")
cruce4_1 = pd.merge(dfe1, dfe2, how='outer', on='C')
cruce4_2 = pd.merge(cruce4_1, dfe3, how='right', on='A')
cruce4_2

Ej. 4:


,A,B,C,Q_x,W,X,D,Q_y,M
0,A10,B10,C10,Q10,W10,X10,D10,Q10,M10
1,A11,B11,C11,Q11,W11,X11,D11,Q11,M11
2,A12,B12,C12,Q12,W12,X12,D12,Q12,M12
3,A13,B13,C13,Q13,W13,X13,D13,Q13,M13
4,A14,B14,C14,Q14,W14,X14,D14,Q14,M14
5,A15,B15,C15,Q15,W15,X15,D15,Q15,M15
6,A16,B16,C16,Q16,W16,X16,D16,Q16,M16
7,A17,B17,C17,Q17,W17,X17,D17,Q17,M17
8,A18,B18,C18,Q18,W18,X18,D18,Q18,M18
9,A19,B19,C19,Q19,W19,X19,D19,Q19,M19


In [89]:
# 5. 
print("Ej. 5:")
cruce5_1 = pd.merge(dfe1, dfe2, how='inner', on='C')
cruce5_2 = pd.merge(cruce5_1, dfe3, how='inner', on='A')
cruce5_2

Ej. 5:


,A,B,C,Q_x,W,X,D,Q_y,M
0,A10,B10,C10,Q10,W10,X10,D10,Q10,M10
1,A11,B11,C11,Q11,W11,X11,D11,Q11,M11
2,A12,B12,C12,Q12,W12,X12,D12,Q12,M12
3,A13,B13,C13,Q13,W13,X13,D13,Q13,M13
4,A14,B14,C14,Q14,W14,X14,D14,Q14,M14
5,A15,B15,C15,Q15,W15,X15,D15,Q15,M15
6,A16,B16,C16,Q16,W16,X16,D16,Q16,M16
7,A17,B17,C17,Q17,W17,X17,D17,Q17,M17
8,A18,B18,C18,Q18,W18,X18,D18,Q18,M18
9,A19,B19,C19,Q19,W19,X19,D19,Q19,M19
